# QC and filter seurat object

In [ ]:
library_load <- suppressMessages(
    
    list(
        
        # Seurat 
        library(Seurat), 
        
        # Scran doubletFinder implementation 
        library(scDblFinder), 
        
        # FastMNN implementation 
        library(SeuratWrappers), 
        
        # Data 
        library(tidyverse), 
        
        # Reticulate
        library(reticulate), 
        
        # Biomart 
        library(biomaRt), 
        
        # Plotting 
        library(ggplot2), 
        library(knitr),
        library(kableExtra),
        library(IRdisplay)
        
    )
)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
options(warn=-1)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")
source("bin/seurat_qc.R")
source("bin/seurat_dea.R")

# Parameter settings

In [ ]:
# Files 
so_raw_file <- "data/object/raw.rds"

so_qc_rds <- "data/object/qc.rds"
so_qc_h5ad <- "data/object/qc.h5ad"

# Plotting Theme
ggplot2::theme_set(theme_global_set()) # From project global source()

# Import Seurat object

In [ ]:
so_raw <- readRDS(so_raw_file)

In [ ]:
so_raw$tissue <- factor(so_raw$tissue, levels=c("Myeloid", "Progenitor"))
so_raw$treatment <- factor(so_raw$treatment, levels=c("NaCl", "CpG"))

# Rank plot

Empty droplets were determined with CellRanger V3.0.2 Lun et al., 2019 EmptyDrop heuristic. RNAse activity of granulocytes might be wrongly identified as empty cells by CellRanger.

**Typical Sample** A steep drop-off is indicative of good separation between the cell-associated barcodes and the barcodes associated with empty GEMs. A ideal barcode rank plot has a distincitve shape, which is referred to as a "cliff and knee".  
**Heterogeneous Sample** Heterogeneous populations of cells in a sample result in two "cliff and knee" distributions. However, there should still be clear separation between the bacodes.   
**Compromised Sample** Round curve and lack of steep cliff may indicate low sample quality or loss of single-cell behavior. This can be due to a wetting failure, premature cell lysis, or low cell viability.   
**Compromised Sample** Defined cliff and knee, but the total number of barcodes detected may be lower than expected. This can be caused by a sample clog or inaccurate cell count.   

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
rank_plot_qc_1 <- rank_plot_qc(so_raw, color_color=color$cellranger_class, formular=tissue~sample_rep+treatment)
rank_plot_qc_1

# Filter by CellRanger class

In [ ]:
so_qc <- subset(so_raw, subset=cellranger_class=="Cell")

# Compute cell cycle score

In [ ]:
# Get Seurat cell cycle genes 
cc_genes_seurat_s <- cc.genes.updated.2019$s.genes
cc_genes_seurat_g2m <- cc.genes.updated.2019$g2m.genes

# Get mouse orthologs from human gene simbols
httr::set_config(httr::config(ssl_verifypeer=FALSE))

hgnc_mart <- useMart("ensembl", dataset="hsapiens_gene_ensembl", host="https://dec2021.archive.ensembl.org/")
mm_mart <- useMart("ensembl", dataset="mmusculus_gene_ensembl", host="https://dec2021.archive.ensembl.org/")

cc_genes_seurat_s <- getLDS(attributes=c("hgnc_symbol"), filters="hgnc_symbol", values=cc_genes_seurat_s, mart=hgnc_mart, attributesL=c("mgi_symbol"), martL=mm_mart, uniqueRows=TRUE)
cc_genes_seurat_s <- cc_genes_seurat_s[, 2]
    
cc_genes_seurat_g2m <- getLDS(attributes=c("hgnc_symbol"), filters="hgnc_symbol", values=cc_genes_seurat_g2m, mart=hgnc_mart, attributesL=c("mgi_symbol"), martL=mm_mart, uniqueRows=TRUE)
cc_genes_seurat_g2m <- cc_genes_seurat_g2m[, 2]

# Save cell cycle genes 
saveRDS(list(s=cc_genes_seurat_s, g2m=cc_genes_seurat_g2m), "data/annotation/cell_cycle_seurat/cc_genes.rds")
write.csv(cc_genes_seurat_s, "data/annotation/cell_cycle_seurat/cc_genes_seurat_s.csv")
write.csv(cc_genes_seurat_g2m, "data/annotation/cell_cycle_seurat/cc_genes_seurat_g2m.csv")

# Compute cell cycle score 
so_qc <- CellCycleScoring(so_qc, s.features=cc_genes_seurat_s, g2m.features=cc_genes_seurat_g2m, set.ident=FALSE)

colnames(so_qc@meta.data) <- gsub("Phase", "cc_phase_class", colnames(so_qc@meta.data))
colnames(so_qc@meta.data) <- gsub("S.Score", "msS_RNA", colnames(so_qc@meta.data))
colnames(so_qc@meta.data) <- gsub("G2M.Score", "msG2M_RNA", colnames(so_qc@meta.data)) 
    
so_qc$msCC_diff_RNA <- so_qc$msS_RNA - so_qc$msG2M_RNA
so_qc$cc_phase_class <- factor(so_qc$cc_phase_class, levels=names(color$cc_phase_class))

# SingleR annotation
SingleR identifies marker genes from the reference and uses them to compute assignment score (based on the Spearman correlation across markers) for each cell in the test dataset against each label in the reference. The label with the highest doublet_stat is the assigned to the test cell, possibly with further fine-tuning to resolve closely related labels.  

first.labels: Labels before fine-tuning  
labels: Labels after fine-tuning  
pruning: labels after pruning  

## ImmGen

In [ ]:
if(FALSE) {
    
    # Load reference data 
    ref <- ImmGenData()
    
    # Seurat object to SingleCellExperiment
    sce <- SingleCellExperiment(list(counts=so_qc@assays$RNA@counts))

    # # Predict labels
    label_main <- SingleR::SingleR(test=sce, ref=ref, labels=ref$label.main, assay.type.test="counts", de.method="classic")
    label_fine <- SingleR::SingleR(test=sce, ref=ref, labels=ref$label.fine, assay.type.test="counts", de.method="classic")

    saveRDS(label_main, "result/singler/label_main_immgen.rds")
    saveRDS(label_fine, "result/singler/label_fine_immgen.rds")

} else {
    
    label_main <- readRDS("result/singler/label_main_immgen.rds")
    label_fine <- readRDS("result/singler/label_fine_immgen.rds")
    
}

# Add labels to Seurat object
label_main_meta <- as.data.frame(label_main) %>% dplyr::select(pruned.labels, tuning.scores.first) %>% dplyr::rename(label_main_immgen=pruned.labels, delta_score_main_immgen=tuning.scores.first)
label_fine_meta <- as.data.frame(label_fine) %>% dplyr::select(pruned.labels, tuning.scores.first) %>% dplyr::rename(label_fine_immgen=pruned.labels, delta_score_fine_immgen=tuning.scores.first)

so_qc <- AddMetaData(so_qc, cbind(label_main_meta, label_fine_meta))

## Haemosphere

In [ ]:
if(FALSE) {
    
    # Load reference data 
    ref <- readRDS("data/haemosphere/se_haemosphere.rds")
    
    # Seurat object to SingleCellExperiment
    sce <- SingleCellExperiment(list(counts=so_qc@assays$RNA@counts))

    # # Predict labels
    label_main <- SingleR::SingleR(test=sce, ref=ref, labels=ref$label.main, assay.type.test="counts", de.method="classic")
    label_fine <- SingleR::SingleR(test=sce, ref=ref, labels=ref$label.fine, assay.type.test="counts", de.method="classic")

    saveRDS(label_main, "result/singler/label_main_haemosphere.rds")
    saveRDS(label_fine, "result/singler/label_fine_haemosphere.rds")

} else {
    
    label_main <- readRDS("result/singler/label_main_haemosphere.rds")
    label_fine <- readRDS("result/singler/label_fine_haemosphere.rds")
    
}

# Add labels to Seurat object
label_main_meta <- as.data.frame(label_main) %>% dplyr::select(pruned.labels, tuning.scores.first) %>% dplyr::rename(label_main_haemosphere=pruned.labels, delta_score_main_haemosphere=tuning.scores.first)
label_fine_meta <- as.data.frame(label_fine) %>% dplyr::select(pruned.labels, tuning.scores.first) %>% dplyr::rename(label_fine_haemosphere=pruned.labels, delta_score_fine_haemosphere=tuning.scores.first)

so_qc <- AddMetaData(so_qc, cbind(label_main_meta, label_fine_meta))

# Doublet detection on pre-filtered cells 

I first tried the strategy from [Sala et al., 2019](https://www.nature.com/articles/s41586-019-0933-9) which was also used in [Barile et al., 2021](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-021-02414-y) with the source documented [here](https://github.com/MarioniLab/EmbryoTimecourse2018/blob/master/analysis_scripts/atlas/5_doublets/doublets.Rmd). We only do the refined method with combined samples. The functions used in the script are documented [here](https://github.com/MarioniLab/EmbryoTimecourse2018/blob/master/analysis_scripts/atlas/core_functions.R). We do doublet detection on sample groups (Myeloid and Progenitor cells of a replicated) and integrated those groups per batch and then by treatment with FastMNN. However, I default later back to just use the standard approach. 

In [ ]:
qc_class_set <- function(so) {
    
    print(so)
    
    # QC filter for all cells
    so$pMt_RNA_min <- -Inf
    so$pMt_RNA_max <- 5
    
    so$nCount_RNA_min <- 1500
    so$nCount_RNA_max <- max(so$nCount_RNA)

    so$qc_class <- ifelse(
        so$cellranger_class=="Cell" & 
        so$nCount_RNA <= so$nCount_RNA_max & 
        so$nCount_RNA > so$nCount_RNA_min &
        so$pMt_RNA <= so$pMt_RNA_max &
        so$pMt_RNA > so$pMt_RNA_min,
        "pass", "fail"
      )
    
    
    so <- subset(so, subset=qc_class=="pass")
    
    print(so)
    
    return(so)

}

In [ ]:
so_db <- Seurat::SplitObject(so_qc, split.by="sample_group")
so_db <- lapply(so_db, qc_class_set)

## Run scDblFinder on pre-filtered data 

In [ ]:
# Compute doublet density per sample group 
so_db <- lapply(so_db, function(so) {
    
    DefaultAssay(so) <- "RNA"
    
    # Remove empty genes from split 
    cnt <- GetAssayData(so, assay="RNA", slot="counts")
    cnt <- cnt[rowSums(cnt) > 0, ]
    
    # Compute doublet density 
    sce <- scDblFinder::scDblFinder(cnt)
    so$doublet_score <- sce$scDblFinder.score
    so$doublet_score_log2 <- log2(sce$scDblFinder.score)
    so$doublet_class <- sce$scDblFinder.class
    
    return(so)
    
}
               )

In [ ]:
options(repr.plot.width=30, repr.plot.height=5)

# Compute sub-cluster per sample group 
so_db <- lapply(so_db, function(so) {
    
    # Cluster function     
    so <- ScaleData(so, features=rownames(so), assay="RNA", verbose=FALSE)
    so <- FindVariableFeatures(so)
    so <- RunPCA(so, npcs=50, features=VariableFeatures(so), assay="RNA", verbose=FALSE)
    so <- FindNeighbors(so, dims=1:50, k.param=15, reduction="pca", verbose=FALSE)
    so <- FindClusters(so, verbose=FALSE, resolution=1, algorithm=1)
    so <- RunUMAP(so, dims=1:50, n.neighbors=30, verbose=FALSE)
    
    dplot_1 <- dplot(so, reduction="umap", group_by="treatment") + scale_color_manual(values=color$treatment) + ggtitle("Treatment")
    dplot_2 <- dplot(so, reduction="umap", group_by="tissue") + scale_color_manual(values=color$tissue) + ggtitle("Tissue")
    dplot_3 <- dplot(so, reduction="umap", group_by="doublet_class") + ggtitle("doublet_class")
    fplot_1 <- fplot(so, reduction="umap", features="doublet_score") + scale_colour_viridis_c(option="inferno") + ggtitle("doublet_score")
    fplot_2 <- fplot(so, reduction="umap", features="doublet_score_log2") + scale_colour_viridis_c(option="inferno") + ggtitle("doublet_score_log2")
    
    dplot_comb <- dplot_1 + dplot_2 + dplot_3 + fplot_1 + fplot_2 + plot_layout(ncol=5)
    plot(dplot_comb)
    
    return(so)
    
}
               )

## Add scDblFinder results to unfiltered Seurat object 

In [ ]:
so_db <- merge(so_db[[1]], so_db[2:length(so_db)])
so_qc <- AddMetaData(so_qc, dplyr::select(so_db@meta.data, doublet_class, doublet_score, doublet_score_log2))

## Set QC filter 

In [ ]:
qc_class_set <- function(so) {
    
    # QC filter for all cells
    so$pMt_RNA_min <- -Inf
    so$pMt_RNA_max <- 5
    
    so$nCount_RNA_min <- 1500
    so$nCount_RNA_max <- max(so$nCount_RNA)

    so$qc_class <- ifelse(
        so$cellranger_class == "Cell" & 
        so$nCount_RNA <= so$nCount_RNA_max & 
        so$nCount_RNA > so$nCount_RNA_min &
        so$pMt_RNA <= so$pMt_RNA_max &
        so$pMt_RNA > so$pMt_RNA_min & 
        so$doublet_class == "singlet", 
        "pass", "fail"
      )
    
    # Remove Ery from Myeloid sort before computing nFeature_RNA
    if(so$tissue[1]=="Myeloid") {
        
        so$qc_class <- ifelse(
            so$label_main_haemosphere=="Ery" | so$label_fine_haemosphere %in% c("preCFUE", "CFUE", "pbEry", "poEry", "Retic") | so$pHb_RNA >= 5, 
            "fail", so$qc_class
        )
        
    }
    
    so_tmp <- subset(so, subset=qc_class=="pass")
    
    so$nFeature_RNA_max <- max(so_tmp$nFeature_RNA)
    so$nFeature_RNA_min <- quantile(so_tmp$nFeature_RNA, 0.01)
    
    so$qc_class <- ifelse(
        so$qc_class == "pass" &
        so$nFeature_RNA <= so$nFeature_RNA_max & 
        so$nFeature_RNA > so$nFeature_RNA_min, 
        "pass", "fail"
      )
    
    return(so)

}

In [ ]:
so_qc <- Seurat::SplitObject(so_qc, split.by="sample_name")
so_qc <- lapply(so_qc, qc_class_set)
so_qc <- merge(so_qc[[1]], so_qc[2:length(so_qc)])

# QC plots 

## Density plot

In [ ]:
density_plot_qc_1 <- density_plot_qc(so=so_qc, title="Density plot UMI count", x=nCount_RNA, xlab="log10(UMI count)", min=nCount_RNA_min, max=nCount_RNA_max, fill_color=color$tissue)
density_plot_qc_2 <- density_plot_qc(so=so_qc, title="Density plot Feature count", x=nFeature_RNA, xlab="log10(Feature count)", min=nFeature_RNA_min, max=nFeature_RNA_max, fill_color=color$tissue)
density_plot_qc_3 <- density_plot_qc(so=so_qc, title="Density plot Mt %", x=pMt_RNA, xlab="Mt [%]", min=0, max=pMt_RNA_max, log10=FALSE, fill_color=color$tissue)

In [ ]:
options(repr.plot.width=22.5, repr.plot.height=5)
density_plot_qc_1 + density_plot_qc_2 + density_plot_qc_3 + plot_layout(nrow=1) & theme(legend.position="bottom")

## Scattern plot

In [ ]:
scattern_plot_qc_1 <- scattern_plot_qc(so=so_qc, title="Mitochondrial gene percentage", color=pMt_RNA)
scattern_plot_qc_2 <- scattern_plot_qc(so=so_qc, title="Hemoglobin gene percentage", color=pHb_RNA)
scattern_plot_qc_3 <- scattern_plot_qc(so=so_qc, title="Ribsonmal gene percentage", color=pRb_RNA)
scattern_plot_qc_4 <- scattern_plot_qc(so=so_qc, title="Fine labels", color=label_fine_haemosphere) + scale_color_manual(values=color$label_fine_haemosphere)
scattern_plot_qc_5 <- scattern_plot_qc(so=so_qc, title="Doublet class", color=doublet_class) + scale_color_manual(values=c("doublet"="#132B43", "singlet"="#56B1F7"))
scattern_plot_qc_6 <- scattern_plot_qc(so=so_qc, title="QC class", color=qc_class) + scale_color_manual(values=c("fail"="#132B43", "pass"="#56B1F7"))

In [ ]:
options(repr.plot.width=22.5, repr.plot.height=10)
scattern_plot_qc_1 + scattern_plot_qc_2 + scattern_plot_qc_3 + scattern_plot_qc_4 + scattern_plot_qc_5 + scattern_plot_qc_6 + plot_layout(ncol=3) & theme(legend.position="bottom")

## Box plots 

In [ ]:
box_plot_qc_1 <- box_plot_qc(so=so_qc, y=nCount_RNA, fill=tissue, ylab="UMI [count]", ymin=0)
box_plot_qc_2 <- box_plot_qc(so=so_qc, y=nFeature_RNA, fill=tissue, ylab="Feature [count]", ymin=0)
box_plot_qc_3 <- box_plot_qc(so=so_qc, y=pMt_RNA, fill=tissue, ylab="Mt [%]", ymin=0, ymax=100)
box_plot_qc_4 <- box_plot_qc(so=so_qc, y=pHb_RNA, fill=tissue, ylab="Hb [%]", ymin=0, ymax=100)
box_plot_qc_5 <- box_plot_qc(so=so_qc, y=pRb_RNA, fill=tissue, ylab="Rbl [%]", ymin=0, ymax=100)

In [ ]:
options(repr.plot.width=22.5, repr.plot.height=20)
box_plot_qc_1[[1]] + box_plot_qc_1[[2]] + box_plot_qc_2[[1]]  + box_plot_qc_2[[2]] + 
box_plot_qc_3[[1]] + box_plot_qc_3[[2]] + box_plot_qc_4[[1]]  + box_plot_qc_4[[2]] + 
box_plot_qc_5[[1]] + box_plot_qc_5[[2]] + plot_spacer() + plot_spacer() + plot_layout(guides="collect", ncol=2) & theme(legend.position="none")

# Filter by QC class

In [ ]:
so_qc <- subset(so_qc, subset=qc_class=="pass")

# UMAP and clustering of all samples on all genes

In [ ]:
so_qc <- NormalizeData(so_qc, assay="RNA")
so_qc <- ScaleData(so_qc, assay="RNA", verbose=FALSE)
so_qc <- RunPCA(so_qc, npcs=50, assay="RNA", features=rownames(so_qc), verbose=FALSE)
so_qc <- FindNeighbors(so_qc, dims=1:30, k.param=20, verbose=FALSE)
so_qc <- FindClusters(so_qc, verbose=FALSE, resolution=1, algorithm=1, group.singletons=FALSE)
so_qc <- RunUMAP(so_qc, dims=1:50, n.neighbors=30, verbose=FALSE)

In [ ]:
options(repr.plot.width=22.5, repr.plot.height=15)

dplot_1 <- dplot(so_qc, reduction="umap", group_by="seurat_clusters", label=TRUE) + ggtitle("Louvain cluster")
dplot_2 <- dplot(so_qc, reduction="umap", group_by="tissue") + scale_color_manual(values=color$tissue) + ggtitle("Tissue")
dplot_3 <- dplot(so_qc, reduction="umap", group_by="treatment") + scale_color_manual(values=color$treatment) + ggtitle("Treatment")
dplot_4 <- dplot(so_qc, reduction="umap", group_by="cc_phase_class") + scale_color_manual(values=color$cc_phase_class) + ggtitle("Cell cycle")
dplot_5 <- dplot(so_qc, reduction="umap", group_by="label_main_haemosphere") + scale_color_manual(values=color$label_main_haemosphere) + ggtitle("Haemosphere label (main)")
dplot_6 <- dplot(so_qc, reduction="umap", group_by="label_fine_haemosphere") + scale_color_manual(values=color$label_fine_haemosphere) + ggtitle("Haemosphere label (fine)")
dplot_7 <- dplot(so_qc, reduction="umap", group_by="sample_rep") + ggtitle("Replicate")
fplot_1 <- fplot(so_qc, reduction="umap", features="Trac") + scale_colour_viridis_c(option="inferno") + ggtitle("Trac")
fplot_2 <- fplot(so_qc, reduction="umap", features="Igkc") + scale_colour_viridis_c(option="inferno") + ggtitle("Igkc")

dplot_comb <- dplot_1 + dplot_2 + dplot_3 + dplot_4 + dplot_5 + dplot_6 + dplot_7 + fplot_1 + fplot_2 + plot_layout(ncol=3)
dplot_comb

In [ ]:
so_qc@meta.data %>% dplyr::group_by(seurat_clusters, treatment, sample_rep) %>% 
    dplyr::summarise(n=n()) %>% data.frame() %>% 
    tidyr::spread(seurat_clusters, n) %>% 
    kableExtra::kable("html") %>% as.character() %>% IRdisplay::display_html()

In [ ]:
marker_22 <- FindMarkers(so_qc, ident.1="22", logfc.threshold=0.5, min.pct=0.01, only.pos=FALSE)
marker_23 <- FindMarkers(so_qc, ident.1="23", logfc.threshold=0.5, min.pct=0.01, only.pos=FALSE)
marker_25 <- FindMarkers(so_qc, ident.1="25", logfc.threshold=0.5, min.pct=0.01, only.pos=FALSE)

In [ ]:
options(repr.plot.width=15, repr.plot.height=5)

cluster_marker <- list(marker_22, marker_23, marker_25)

vp_cluster_marker <- lapply(seq_along(cluster_marker), function(i) vp_dea(cluster_marker[[i]], title=c("Cluster 22", "Cluster 23", "Cluster 25")[i], log2_thold=0.25))
wrap_plots(vp_cluster_marker) 

# Remove T cell (Trac) and Plasma cell (Igkc) cluster

In [ ]:
so_qc <- subset(so_qc, subset=seurat_clusters!=22)
so_qc <- subset(so_qc, subset=seurat_clusters!=23)
so_qc <- subset(so_qc, subset=seurat_clusters!=25)

# Save result objso_qc_rds

In [ ]:
# Store data as RDS 
saveRDS(so_qc, so_qc_rds)

In [ ]:
# Store data as h5ad 
adata <- import("anndata", as="adata", convert=FALSE)
pd <- import("pandas", as="pd", convert=FALSE)
np <- import("numpy", as="np", convert=FALSE)

    
# Transform dgCMatrix to 
X <- GetAssayData(so_qc, assay="RNA", slot="counts") %>% as.matrix() %>% t()
X <- np$array(X, dtype=np$int32)
    
adata <- adata$AnnData(X=X, obs=so_qc@meta.data)
adata$var_names <- rownames(GetAssayData(so_qc, assay="RNA", slot="counts"))

adata$raw <- adata
adata$write_h5ad(so_qc_h5ad)

# Session info

In [ ]:
sessionInfo()